Breast Cancer diagnosis

In [ ]:
import numpy as np 
import pandas as pd 
df = pd.read_csv('../input/data.csv')
df.head()

In [ ]:
# Check the data size
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(df.columns[32], axis=1, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#%matplotlib inline

sns.pairplot(df, vars=df.columns[2:5], hue='diagnosis', markers=['o', 's'], size=3, kind='reg')
plt.show()

We can see some linear relationships from the above graph between radius-perimeter-area. (NOTE that I am trying for a few hours to upload/copy my notebook on kaggle and it just doesn't work.So, I have used only 3 columns from the data! instead of all data...)

In [ ]:
# Use dummies in order to map the diagnosis labels to 0 and 1
df_new = pd.get_dummies(df)
df_new.head()

Plot a correlation matrix.

In [ ]:
cols = list(df_new.columns)
fig, ax = plt.subplots(figsize=(26,20))
# We take values[:, 1:] in order to ommit the id's.
cm = np.corrcoef(df_new.values[:, 1:].T)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size':8}, yticklabels=cols[1:], xticklabels=cols[1:])
plt.show()

From the above we can see that the area-perimeter-radius-concave_points are strongly correlated to the malignant case.

In [ ]:
# Let's get back to the original data
df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.metrics import accuracy_score

# Take every line except from the columns with the id's and the labelled data (benigh, malignant)
X = df.values[:, 2:].astype(float)
# The labelled data
y = df.values[:, 1]

We are going to test different models and check their metrics.

In [ ]:
# Prepare the models
lr = LogisticRegression(C=1000.0, random_state=0)
svm_linear = SVC(kernel='linear', C=1.0, random_state=0, probability=True)
svm_nonlinear = SVC(kernel='rbf', C=10.0, gamma = 0.1, random_state=0, probability=True)
random_forest = RandomForestClassifier(criterion='entropy', random_state=0)
gaussian_nb = GaussianNB()

In [ ]:
labels = ['Logistic Regression', 'SVC linear', 'SVC non linear', 'Random Forest', 'Naive Bayes']
models = [lr, svm_linear, svm_nonlinear, random_forest, gaussian_nb]
# Apply cross validation score
for model, label in zip(models, labels):
    scores = model_selection.cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print('Mean: {0}\t Std: {1}\t Label: {2}'.format(scores.mean(), scores.std(), label))

We can see that most models have similar accuracy except from the SVC non linear.
Let's just test what about the voting classifier which aggregates the predictions of each classifier and predicts the class that gets the most votes.

In [ ]:
voting = VotingClassifier(estimators=[('lr', lr), ('svm_linear', svm_linear), ('svm_nonlinear', svm_nonlinear), 
                                      ('random_forest', random_forest), ('gaussian_nb', gaussian_nb)],
                        voting = 'soft', weights=[1,1,1,1,1])

labels = ['Logistic Regression', 'SVC linear', 'SVC non linear', 'Random Forest', 'Naive Bayes', 'Voting']
models = [lr, svm_linear, svm_nonlinear, random_forest, gaussian_nb, voting]

for model, label in zip(models, labels):
    scores = model_selection.cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print('Mean: {0}\t Std: {1}\t Label: {2}'.format(scores.mean(), scores.std(), label))

We saw that voting is almost equal to random forest classifier.
we are going to use the random forest for training our data.

In [ ]:
# Split data into training and test datasets.
# Playing with the test size, I found that 20% gives the best results
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Standardize the features in order to have properties of standar normal distribution (mean = 0, std = 1)
std_sc = StandardScaler()
X_train = std_sc.fit_transform(X_train)
X_test = std_sc.fit_transform(X_test)

In [ ]:
# Train our model
random_forest.fit(X_train, y_train)
# Get the feature importances attribute
importances = random_forest.feature_importances_
indices = np.argsort(importances)[::-1]

# Plot feature importances
fig, axes = plt.subplots(figsize=(14, 10))
plt.bar(range(X_train.shape[1]), importances[indices], align='center')
plt.title('Feature Importances')
plt.xticks(range(X_train.shape[1]), df.columns[2:], rotation=90)
plt.show()

We can see that the most favored feature is the radius mean.We remember from the correlation matrix that the mean radius is strongly correlated to perimeter, texture and area features.The least favored is the fractal dimension.Keep in mind that feature importances are normalized so that they sum up to 1.0.

In [ ]:
# Predict class
y_pred = random_forest.predict(X_test)
print('Accuracy: {0}'.format(accuracy_score(y_test, y_pred)))

Ok!Nice accuracy using random forest classifier!